# 토픽모델링
2_Word_Cloud 실습의 같은 빅카인즈 데이터로 토픽모델링을 실습합니다.

## 개념
토픽모델링의 개념은 [ratsgo's blog](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/06/01/LDA/)를 참조로 설명합니다  
<br>
토픽모델링은 기계학습 알고리즘입니다.  
기계가 알고리즘에 따라 텍스트의 단어 분포로부터 추상적인 '토픽'을 스스로 찾아나가는 과정입니다.  
텍스트 데이터가 풍부할수록 결과가 인간에게 보다 유의미할 가능성이 높습니다

## 데이터 읽기
다양한 라이브러리를 본다는 취지에서 2_Word_Cloud에서 사용한 pandas 라이브러리와 달리  
openpyxl이라는 라이브러리를 사용합니다.

In [ ]:
from openpyxl import load_workbook
from collections import defaultdict
Theme="President" # 분석 목적과 데이터에 맞게 테마 설정
File='data\\NewsResult_20211126-20220226.xlsx' # 실제 데이터 파일 이름 입력
wb = load_workbook(File)
ws = wb.active

앞서 pandas에서 열 순서를 기준으로 키워드 데이터를 취했다면  
이번 경우 엑셀의 열을 기준으로 취합니다.(엑셀 프로그램으로 파일을 열면 *키워드*는 *O*열에 위치)

In [ ]:
# 키워드(칼럼 O)의 단어들 리스트로 읽기
words = ws['O']
words_list = [x.value.split(',') for x in words[1:]]

In [ ]:
words_list[0]

## 전처리
단순화 하여 전체 텍스트에서 한번 나타난 단어만 제거  
토픽모델링이란 분석 방법의 특성상 한번만 등장한 단어는 토픽 학습에 도움이 되지 못합니다

In [ ]:
# 한번만 나타난 단어는 제거
frequency = defaultdict(int)
for text in words_list:
    for token in text:
        frequency[token] += 1
words_list = [[token for token in text if frequency[token] > 1] for text in words_list]

## [Gensim](https://radimrehurek.com/gensim/)
**Gensim**은 토픽모델링을 비롯해 워드엠베딩 등<br> 
다양한 자연어처리 기계학습 알고리즘을 제공하는 유용한 라이브러리입니다<br>
이 라이브러리를 활용해 LDA 모델을 만들어 보겠습니다

In [ ]:
import logging #학습 과정을 보기 위함
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim import corpora
from gensim.models import LdaModel

### 파라미터 설정
토픽모델링에서 토픽의 숫자는 분석자가 정하는 하이퍼파라미터입니다.<br>
해당 말뭉치에 가장 적절한 토픽의 숫자는 몇개일까요? 쉽게 정하기 어렵습니다.<br>
추정하는 방법들이 있는데 대표적인 것이 **Perplexity**와 **Coherence** 점수입니다([참조 코어닷 블로그](https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/))<br>
지금은 임의의 숫자로 설정합니다

In [ ]:
'''
하이퍼 파라미터 설정
'''
K = 10 # 토픽수
iterations = 1000 # 반복 횟수
random_state = 1 # 랜덤 값을 임의로 고정. 이 값을 고정해야 뒤에 같은 토픽 재현이 가능함.
fname="lda_"+Theme+"K"+str(K)+"R"+str(random_state)+"I"+str(iterations) #임의의 모델명

우선 사전을 구축

In [ ]:
dictionary = corpora.Dictionary(words_list)

말뭉치를 학습에 적절한 **Bag of Words(BOW) 모델** 형태로 변환

In [ ]:
corpus = [dictionary.doc2bow(text) for text in words_list]

### 모델 구축

In [ ]:
lda = LdaModel(corpus, 
               num_topics=K, 
               id2word=dictionary,
               random_state=random_state, 
               iterations=iterations)

### 결과 보기 및 저장
인간에게 의미가 큰 토픽 모델은 각 모델이 겹치는 단어가 적고<br>
토픽들이 명확한 개성을 가져 해석하기 쉬운 것입니다

In [ ]:
lda.print_topics(-1,20) #전체 토픽(-1)의 비중 상위 20개 단어를 봅니다

#### 결과 텍스트 파일로 저장

In [ ]:
topics = lda.print_topics(-1,20)
feat_fname = fname+'_feats.txt'
with open(feat_fname, 'w') as text_file:
    for topic_num, features in topics:
        text_file.write("Topic={0} \n {1} \n".format(topic_num, features))

### 모델 저장
뒤에 다시 모델을 불러낼 일이 있을 것 같다면 파일로 저장해 둡니다.  
다시 학습할 필요 없이 모델을 로드해서 사용할 수 있습니다.

In [ ]:
dictionary.save(fname+'_dictionary.pkl') #DICT_PATH
corpora.MmCorpus.serialize(fname+'_corpus.mm', corpus) #CORPUS_PATH
lda.save(fname) #MODEL_PATH

In [ ]:
'''로딩하기

loaded_dict = corpora.Dictionary.load(DICT_PATH)
loaded_corp = corpora.MmCorpus(CORPUS_PATH)
lda = LdaModel.load(MODEL_PATH)
'''

## 시각화
시각화는 토픽 모델링을 살펴보는 좋은 방법입니다.  

In [ ]:
from pyLDAvis import gensim_models,display
import matplotlib.pyplot as plt

In [ ]:
vis = gensim_models.prepare(lda,corpus,dictionary,sort_topics=False)

In [ ]:
pyLDAvis.display(vis)

In [ ]:
# 파일로 저장하기
pyLDAvis.save_html(vis, fname+'.html')